In [1]:
from flowmapper.utils import places
import randonneur_data as rd
import randonneur as rn
from pathlib import Path
import bw2data as bd

In [2]:
bd.projects.set_current("ecoinvent-multiversion-cutoff-base")

In [3]:
mapping = {(obj['name'], tuple(obj['categories']), obj['unit']): obj['code'] for obj in bd.Database("ecoinvent-3.10-biosphere")}

In [4]:
registry = rd.Registry()

In [5]:
def categories(o):
    if o[1] == "(unspecified)":
        return [o[0]]
    return o

In [6]:
data = [
    {
        "source": {"name": f"Water/m3, {place}", "unit": "m3", "context": context_pair['source']['context']},
        "target": {
            "name": 'Water', 
            "context": context_pair['target']['context'], 
            'identifier': mapping[('Water', tuple(categories(context_pair['target']['context'])), 'cubic meter')]
        },
        "location": place,
        "comment": "Manual matching of all 'Water/m3, <place>' patterns"
    }
    for place in places
    for context_pair in registry.get_file('simapro-9-ecoinvent-3-context')['replace']
    if context_pair['target']['context'][0] in ('air', 'water')
    and ('Water', tuple(categories(context_pair['target']['context'])), 'cubic meter') in mapping
] + [
    {
        "source": {"name": f"Water/m3, {place}", "unit": "m3", "context": context_pair['source']['context']},
        "target": {
            "name": "Water, unspecified natural origin", 
            "context": context_pair['target']['context'], 
            "identifier": mapping[('Water, unspecified natural origin', tuple(categories(context_pair['target']['context'])), 'cubic meter')]
        },
        "location": place,
        "comment": "Manual matching of all 'Water/m3, <place>' patterns"
    }
    for place in places
    for context_pair in registry.get_file('simapro-9-ecoinvent-3-context')['replace']
    if context_pair['target']['context'][0] == 'natural resource'
    and ('Water, unspecified natural origin', tuple(categories(context_pair['target']['context'])), 'cubic meter') in mapping
] + [
    {
        "source": {"name": f"Water/m3, {place}", "unit": "m3", "context": context_pair['source']['context']},
        "target": {
            "name": "Water, unspecified natural origin", 
            "context": ('natural resource', 'in water'), 
            "identifier": mapping[('Water, unspecified natural origin', ('natural resource', 'in water'), 'cubic meter')]
        },
        "location": place,
        "comment": "Manual matching of all 'Water/m3, <place>' patterns"
    }
    for place in places
    for context_pair in registry.get_file('simapro-9-ecoinvent-3-context')['replace']
    if context_pair['target']['context'][0] == 'natural resource'
    and (len(context_pair['source']['context']) == 1 or (context_pair['source']['context'][1] == '(unspecified)'))
]

In [7]:
for context_pair in registry.get_file('simapro-9-ecoinvent-3-context')['replace']:
    if ('Water', tuple(categories(context_pair['target']['context'])), 'cubic meter') not in mapping and context_pair['target']['context'][0] in ('air', 'water'):
        print(context_pair['target']['context'])
for context_pair in registry.get_file('simapro-9-ecoinvent-3-context')['replace']:
    if ('Water, unspecified natural origin', tuple(categories(context_pair['target']['context'])), 'cubic meter') not in mapping and context_pair['target']['context'][0] == 'natural resource':
        print(context_pair['target']['context'])        

['air', 'indoor']
['air', 'indoor']
['air', 'indoor']
['natural resource', '(unspecified)']
['natural resource', '(unspecified)']
['natural resource', '(unspecified)']
['natural resource', 'biotic']
['natural resource', 'biotic']
['natural resource', 'biotic']
['natural resource', 'in air']
['natural resource', 'in air']
['natural resource', 'in air']
['natural resource', 'land']
['natural resource', 'land']
['natural resource', 'land']
['natural resource', '(unspecified)']
['natural resource', '(unspecified)']
['natural resource', '(unspecified)']
['natural resource', '(unspecified)']
['natural resource', '(unspecified)']
['natural resource', '(unspecified)']
['natural resource', '(unspecified)']
['natural resource', '(unspecified)']
['natural resource', '(unspecified)']


In [8]:
dp = rn.Datapackage(
    name="simapro-9-ecoinvent-3-water-slash-m3",
    description="Specific mapping to handle regionalized 'Water/m3, <place>' pattern for all possible locations",
    contributors=[
        {"title": "Chris Mutel", "path": "https://chris.mutel.org/", "role": "author"},
    ],
    mapping_source=rn.MappingConstants.SIMAPRO_CSV | {"as-tuple": ["context"]},
    mapping_target=rn.MappingConstants.ECOSPOLD2_BIO_FLOWMAPPER | {"as-tuple": ["context"]},
    version="1.0.0",
    source_id="SimaPro-9",
    target_id="ecoinvent-3-biosphere"
)
dp.add_data("replace", data)
registry.add_file(dp.to_json(Path("simapro-9-ecoinvent-3-water-slash-m3.json")), replace=True)

PosixPath('/Users/cmutel/Code/randonneur_data/randonneur_data/data/simapro-9-ecoinvent-3-water-slash-m3.xz')